# Base NPS - Loja de Ferragens TOliveira S.A.

        Aqui esta como foi gerado nossa base NPS e um pouco de como importamos alguns arquivos xlsx!

## Objetivo

        Este notebook tem como objetivo demonstrar os conhecimentos que adquiri nos últimos meses de treinamento com pandas, aqui você irá encontrar conhecimentos de:
        - Criação de DF;
        - Manipulação de DF;
        - Importação e Exportação de DT;
        - Geração de números randômicos (binomial, normal e linear).
        

### Primeiros passos
        Vamos importar as nossas libs necessárias e também as planilhas que tem dentro do nosso arquivo Excel para entendermos melhor o que teremos dentro desse arquivo.

In [46]:
import pandas as pd 
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
from scipy.stats import binom

# resgatamos os nomes das abas do arquivo que possuem dados do NPS, no caso puxaremos os dados de JUN, JUL e AGO
SN = pd.ExcelFile(r'C:\Users\tthia\Desktop\NPS\BASE_NPS.xlsx')
SN.sheet_names 

['NPSJUN', 'NPSJUL', 'NPSAGO', 'Plan2']

### Importação de nossos dataframes
        Vamos agora importar nossos dataframes de cada planilha do Excel que possui nome do mês assim como descrito no README da base.

In [47]:
# importa cada planilha do arquivo 
df1 = pd.read_excel(r'C:\Users\tthia\Desktop\NPS\BASE_NPS.xlsx', sheet_name='NPSJUN', converters = {'ID Cliente':str})
df1 ['Mês'] = '06/2020'

df2 = pd.read_excel(r'C:\Users\tthia\Desktop\NPS\BASE_NPS.xlsx', sheet_name='NPSJUL', converters = {'ID Cliente':str})
df2 ['Mês'] = '07/2020'

df3 = pd.read_excel(r'C:\Users\tthia\Desktop\NPS\BASE_NPS.xlsx', sheet_name='NPSAGO', converters = {'ID Cliente':str})
df3 ['Mês'] = '08/2020'

# empilha todas planilhas
df = pd.concat([df1, df2, df3])

df.tail()

,ID Cliente,Mês,ID
18288,038448,08/2020,NaN
18289,050899,08/2020,NaN
18290,070355,08/2020,NaN
18291,011620,08/2020,NaN
18292,012211,08/2020,NaN


        Notou que temos duas colunas de ID? Porém uma não está preenchida, então ao olhar no README vemos que essa planilha teve um erro durante sua criação e veio com o nome errado, para isso iremos renomea-la para o nome correto e assim por fim empilha-la!

In [48]:
df2 = pd.read_excel (r'caminho\BASE_NPS.xlsx', sheet_name='NPSJUL').rename(columns={"ID": "ID Cliente"})
df2 ['Mês'] = '07/2020'

df = pd.concat([df1, df2, df3])

df.head()

,ID Cliente,Mês
0,000002,06/2020
1,000004,06/2020
2,000014,06/2020
3,000015,06/2020
4,000022,06/2020


        Com posse de nossa base de clientes vamos criar alguns números randomicos para sua nota e categoria, para nota iremos utilizar uma distribuição binomial e para categoria utilizaremos uma distribuição normal, também por termos só 3 meses em nossa base, por que não aumentamos um pouco mais essa base para termos uma maior linha do tempo?

In [56]:
#vamos criar uma base primaria com ID Cliente e Mês, distribuidos linearmente
#para que tenhamos os mesmos valores, irei fixar nosso random por uma semente primária!
np.random.seed(155)

#iniciamos os algoritmos de distribuição do python e distribuimos de o ID Cliente (como temos 100k de clientes colocamos de 
# 0 até 100k e para os meses colocamos de 1 a 6)

rng = np.random.default_rng()
clientes = rng.integers(0, 100000, size = 120000)
meses = rng.integers(1, 6, size = 120000)

#agora criamos nosso df com base nas series que criamos acima 
dados = {'ID Cliente': clientes, 'Mês': meses}
df4 = pd.DataFrame(data=dados)
df4.head()

,ID Cliente,Mês
0,22878,1
1,3203,5
2,58417,1
3,98374,4
4,28809,5


In [57]:
#utilizamos a função map para trocar o nome do mês

meses = {1:"05/2020",
         2:"04/2020",
         3:"03/2020",
         4:"02/2020",
         5:"01/2020"}
df4['Mês'] =df4['Mês'].map(meses)

df4

,ID Cliente,Mês
0,22878,05/2020
1,3203,01/2020
2,58417,05/2020
3,98374,02/2020
4,28809,01/2020
...,...,...
119995,87760,02/2020
119996,5261,01/2020
119997,70086,04/2020
119998,59434,04/2020


In [58]:
#agora juntamos com o df antigo

df = pd.concat([df, df4])
df

,ID Cliente,Mês,Nota,Categoria
0,000002,06/2020,7.0,Preço
1,000004,06/2020,10.0,Preço
2,000014,06/2020,6.0,Entrega
3,000015,06/2020,7.0,Preço
4,000022,06/2020,8.0,Qualidade
...,...,...,...,...
119995,87760,02/2020,NaN,NaN
119996,5261,01/2020,NaN,NaN
119997,70086,04/2020,NaN,NaN
119998,59434,04/2020,NaN,NaN


In [59]:
#criação da coluna 'Nota' com distribuição randomica binomial
df['Nota'] = binom.rvs(n=10,p=0.8,size=len(df))

df.head()

#podemos utilizar a função abaixo para conferir se nosso range de notas realmente está de 0 até 10
#df.agg({'Nota' : [np.min, np.max]})

,ID Cliente,Mês,Nota,Categoria
0,000002,06/2020,8,Preço
1,000004,06/2020,6,Preço
2,000014,06/2020,9,Entrega
3,000015,06/2020,8,Preço
4,000022,06/2020,10,Qualidade


In [60]:
#criação da coluna 'Categoria' com distribuição randomica normal
df['Categoria']  = np.random.normal(loc=2, scale=0.5, size=len(df)).round()

#após distribuir pela normal os dados de categoria, coloquei a função de arredondamento para termos números inteiros e utilizarmos
#um dicionário para classificar (também poderiamos utilizar a função 'map'!)

cat_nome = [
    (df['Categoria'] == 0),
    (df['Categoria'] == 1),
    (df['Categoria'] == 2),
    (df['Categoria'] == 3),
    (df['Categoria'] == 4)]
cat_depara = ['Atendimento','Entrega','Preço', 'Qualidade', 'Sem_Categoria']
df['Categoria'] = np.select(cat_nome, cat_depara)

df.head()

,ID Cliente,Mês,Nota,Categoria
0,000002,06/2020,8,Qualidade
1,000004,06/2020,6,Preço
2,000014,06/2020,9,Preço
3,000015,06/2020,8,Preço
4,000022,06/2020,10,Preço


### Exportação
        Com o arquivo formatado e pronto em mãos, vamos exporta-lo para poder usar em nossa análise de NPS!

In [61]:
df.to_excel(r'caminho\BASE_NPS_DADOS.xlsx', index = False)

### Considerações finais

        Utilizei a curva binomial em "Notas" pois com ela conseguimos ter um intervalo de 0 a 10 de números inteiro e ao utilizar da probabilidade se concentrar em 7 (p=0,7) consigo deixar a curva com flexão bem perto dos nosso intervalos de    NPS para Promotor, Passivo e Detrator, para a "Categoria" utilizei a curva normal para termos uma categoria em evidência em nosso NPS!